# Learning to play Vegas-style Solitaire with AI

---

Author: Andrew Vadnais

Description: Readme in repo

First, some import statements:

In [3]:
import numpy as np

## State

To uniquely describe a single point in the game of Klondike, we need to know a few things.

$\cdot$ the top card in each of the foundation stacks

$\cdot$ the full sequence in each of the playing stacks

$\cdot$ the drawing stack

$\cdot$ the current location in the drawing stack

Therefore, our state class should include all of these things.  In addition, we should include some kind of rating of how good this round is -- by keeping it general, we can investigate the usefulness of certain tactics later on.  We'll also include an extensive representation method in this class, to make any print-outs of the board pretty.

In [155]:
class State:
    def __init__(self,foundation,playing,drawing,drawing_location=0):
        self.found = foundation
        self.play = playing
        self.draw = drawing
        self.loc = drawing_location
        self.known = [x for x in range(7)]
        self.obj = None
        
    def __repr__(self):
        printOut = str(self.found[0])+"h "+str(self.found[1])+"d "+str(self.found[2])+"c "+str(self.found[3])+"s\n"
        
        max_len = np.max([len(stack) for stack in self.play])
        for row in range(max_len):
            row_curr = []
            for stack in range(7):
                if(len(self.play[stack])>row):
                    card = str(self.play[stack][row])
                    if(row<self.known[stack]):
                        card = "---"
                    else:
                        for space in range(3-len(card)):
                            card = " " + card
                    row_curr.append(card)
                else:
                    row_curr.append("   ")
                    
            printOut += str(row_curr[0])+" " +str(row_curr[1])+" " +str(row_curr[2])+" "
            printOut += str(row_curr[3])+" " +str(row_curr[4])+" " +str(row_curr[5])+" "
            printOut += str(row_curr[6])+"\n"
            
        printOut += "          "+str(self.draw[self.loc])+" "+str(self.draw[self.loc+1])+" "+str(self.draw[self.loc+2])+"\n"
        
        printOut += "\nObjective Value: "+str(self.obj)
        return printOut

For the sake of simplicity, we'll define a function here to return a random initial state.

In [164]:
def initial_state(objective_func,found=[0,0,0,0]):
    cards = np.array([str(num)+suit for num in range(1,14) for suit in ['h','c','d','s']])
    cards_shuff = [i for i in range(len(cards))]
    np.random.shuffle(cards_shuff)
    deck_loc = 0
    
    foundation = found
    play = [[] for ii in range(7)]
    
    #dealing under vegas regulations - each new card from the top of the deck
    for row in range(7):
        for stack in range(row,7):
            play[stack].append(cards[cards_shuff[deck_loc]])
            deck_loc += 1

    drawing = cards[cards_shuff[deck_loc:]]
    
    state_init = State(foundation=foundation, playing=play, drawing=drawing)
    state_init.obj = objective_func(state_init)
    
    return state_init

We're gonna create a simple objective function to test the above initial state creation.  We'll just use the number of cards in our foundation stacks (that is, we expect 0 to be returned)

In [166]:
def found_obj(curr):
    return np.sum(curr.found)

initial_state_test = initial_state(found_obj)
print(initial_state_test,"\n")

test_state = initial_state(found_obj,[3,3,4,3])
print(test_state)

0h 0d 0c 0s
 7d --- --- --- --- --- ---
     8s --- --- --- --- ---
         3d --- --- --- ---
            12s --- --- ---
                11h --- ---
                     1c ---
                        12c
          12d 9h 2d

Objective Value: 0 

3h 3d 4c 3s
 1h --- --- --- --- --- ---
     1d --- --- --- --- ---
        11h --- --- --- ---
             2h --- --- ---
                10c --- ---
                     5c ---
                        13s
          5d 7s 1c

Objective Value: 13


## Game

Now that we can 